## Experiment 3

Processing raster images by simulating, visualizing and shifting superposition according to color pixel values of an image instead of defined phases.
This notebook is for further experimentation and adjustments within the scripts.

### Including all modules

In [1]:
# Qiskit for Quantum computation
from qiskit import QuantumCircuit, Aer, transpile, execute
from qiskit.providers.aer.noise import NoiseModel, QuantumError, ReadoutError, pauli_error

# PILlow for image generation
from PIL import Image, ImageColor

# Handy math libraries
import numpy as np
import math

# For saving files in current directory
import os

In [2]:
# Simulate Quantum computation

def run_qc(circuit, backend, output, n_shots):

    # -------------------------------------------------------------------------------
    
    # Build basic bit-flip error noise model

    # Example error probabilities
    p_reset = 0.03
    p_meas = 0.1
    p_gate1 = 0.05

    # QuantumError objects
    error_reset = pauli_error([('X', p_reset), ('I', 1 - p_reset)])
    error_meas = pauli_error([('X',p_meas), ('I', 1 - p_meas)])
    error_gate1 = pauli_error([('X',p_gate1), ('I', 1 - p_gate1)])
    error_gate2 = error_gate1.tensor(error_gate1)

    # Add errors to noise model
    noise_bit_flip = NoiseModel()
    noise_bit_flip.add_all_qubit_quantum_error(error_reset, "reset")
    noise_bit_flip.add_all_qubit_quantum_error(error_meas, "measure")
    noise_bit_flip.add_all_qubit_quantum_error(error_gate1, ["u1", "u2", "u3"])
    noise_bit_flip.add_all_qubit_quantum_error(error_gate2, ["cx"])
    
    # -------------------------------------------------------------------------------
    
    # Execution and options
    
    # Load simulator (Aer)
    backend_simulate = Aer.get_backend('aer_simulator')
    
    # Execute on Aer
    if (backend == 'sim'):
        run = execute(circuit,
                      backend_simulate,
                      shots = n_shots,
                      memory = True).result()
        if (output == 'count'):
            out = run.get_counts()
        if (output == 'memory'):
            out = run.get_memory()
    
    # Execute on Aer + noise model
    if (backend == 'sim_noise'):
        run = execute(circuit,
                      backend_simulate,
                      noise_model=noise_bit_flip,
                      shots = n_shots,
                      memory = True).result()
        if (output == 'count'):
            out = run.get_counts()
        if (output == 'memory'):
            out = run.get_memory()

    return out

In [3]:
# Map output values linear accoring to input values of x

def lin_map(x, in_min, in_max, out_min, out_max):
    if in_min == in_max:
        return out_max / 2
    else:
        return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min

In [4]:
# Retrieving input image data

def img_data(path):

    img = Image.open(path).getdata()
    px_rgb_vals = list(img)
    
    return px_rgb_vals

In [5]:
# Output image generating

def img_generating(data_vals, path):
    a = int(len(data_vals) ** (1/2))
    b = int(len(data_vals) ** (1/2))
    c = int(len(data_vals[0]))
    d = int(len(data_vals[0][0]))
    img = Image.new('RGB', (a * c, b * d))

    for i in range(a):
        for j in range(b):
            for k in range(c):
                for l in range(d):
                    color = int(data_vals[j + (a * i)][k][l] * 255) # [npatch index][npatch col][npatch row]
                    img.putpixel(((k + (j * c)), l + (i * d)), (color, color, color)) # (x, y, rgb color)

    #img.show()
    img.save(path)

In [6]:
# Serial input and output data processing

def serial_qc_processing(data_in, size_npatch, channel):
    a = int(len(data_in) ** (1/2))
    
    data_out = []
    qb = 1

    for i in range(a ** 2):
        c_to_rad = lin_map(data_in[i][channel], 0, 255, 1.5, 0.5)

        # Quantum circuit
        qc = QuantumCircuit(qb)
        qc.reset(0)
        qc.h(0)
        qc.ry(math.pi * c_to_rad, 0)
        qc.measure_all()
        
        # Measurements
        result = run_qc(qc, 'sim_noise', 'memory', size_npatch ** 2)

        # Output data formatting
        result = np.reshape(result, (size_npatch, size_npatch)).astype(float)
        data_out.append(result)
        
    return data_out

In [7]:
# Parallel input and output data processing

def parallel_qc_processing(data_in, size_npatch, channel):
    data_out = []
    qb = len(data_in)

    # Quantum circuit
    qc = QuantumCircuit(qb)
    for i in range(qb):
        c_to_rad = lin_map(data_in[i][channel], 0, 255, 1.5, 0.5)
        qc.reset(i)
        qc.h(i)
        qc.ry(math.pi * c_to_rad, i)
    qc.measure_all()

    # Measurements
    result = run_qc(qc, 'sim_noise', 'memory', size_npatch ** 2)

    # Output data formatting
    for j in range(qb):
        result_form = []
        for k in range(size_npatch ** 2):
            result_form.append(result[k][j]) # ['[0123]']

        result_form = np.reshape(result_form, (size_npatch, size_npatch)).astype(float)
        data_out.append(result_form)

    data_out.reverse()
    
    return data_out

### Execution of functions with different iterative variations

In [8]:
# Single exection of SERIAL_qc_processing with input image

# Input image HAS to be squared
input_img_url = '/Users/niklas/Desktop/X_BA_Thesis/graphics/IPYNB/_input/samples/sample_alphanum_A_inv_64px.png'
resolution = 4

'''
r = 0 
g = 1
b = 2
'''
channel = 0

new_img = img_data(input_img_url)
new_img = serial_qc_processing(new_img, resolution, channel)

# Naming
size = len(new_img)
size = int((size * (resolution ** 2)) ** (1/2))
img_name = os.path.basename(input_img_url)[:-4]

img_generating(new_img, f"{img_name}_serial_{size}x{size}px.png")

In [11]:
# Single exection of PARALLEL_qc_processing with input image

# Input image HAS to be squared
input_img_url = '/Users/niklas/Desktop/X_BA_Thesis/graphics/IPYNB/_input/samples/sample_alphanum_A_inv_8px.png'
resolution = 8

'''
r = 0 
g = 1
b = 2
'''
channel = 0

new_img = img_data(input_img_url)
new_img = parallel_qc_processing(new_img, resolution, channel)

# Naming
size = len(new_img)
size = int((size * (resolution ** 2)) ** (1/2))
img_name = os.path.basename(input_img_url)[:-4]

img_generating(new_img, f"{img_name}_parallel_{size}x{size}px.png")

In [ ]:
# Process multiple images serially

# Name the input *folder*
input_dir = "/Users/niklas/Desktop/out"
resolution = 2

'''
r = 0 
g = 1
b = 2
'''
channel = 0

files = os.listdir(input_dir)

for file in files:
    
    # Images have to be squared and PNG!
    if ('.png' in file):
        new_img = img_data(input_dir + "/" + file)
        new_img = serial_qc_processing(new_img, resolution, channel)

        # Naming
        size = len(new_img)
        size = int((size * (resolution ** 2)) ** (1/2))
        img_name = os.path.basename(file)[:-4]

        img_generating(new_img, f"{img_name}_serial_{size}x{size}px.png")